In [ ]:
import json

#### Read Dataset & Entities

In [2]:
def read_lines(file_path):
    files = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            files.append(line.strip())
    return files

In [3]:
test_source_path = '/home/ml/users/cadencao/XSum/fairseq_files/train.source'
test_target_path = '/home/ml/users/cadencao/XSum/fairseq_files/train.target'

xsum_source = read_lines(test_source_path)
xsum_target = read_lines(test_target_path)

In [4]:
xsum_ents = json.load(open('xsum_train_ents.json', 'r'))

In [5]:
assert len(xsum_source) == len(xsum_target) == len(xsum_ents)
print(len(xsum_target))

203575


#### Build Clean Dataset

In [6]:
from tqdm import tqdm

In [7]:
train_src_clean, train_tgt_clean = [], []
labels = []

for i in tqdm(range(len(xsum_source))):
    tgt_ents = [xsum_target[i][e['start']: e['end']].lower() for e in xsum_ents[i]['ents']]
    
    entiled = True
    for te in tgt_ents:
        if te not in xsum_source[i].lower():
            entiled = False
            labels.append(0.1)
            break
    
    if entiled:
        labels.append(0.9)
        train_src_clean.append(xsum_source[i])
        train_tgt_clean.append(xsum_target[i])

100%|██████████| 203575/203575 [00:02<00:00, 98285.62it/s]


In [8]:
len(train_src_clean)

54019

In [9]:
train_tgt_clean[0]

'Fears that a nearby dam might collapse are reportedly causing panic and confusion in the Syrian city of Raqqa, a stronghold of the so-called Islamic State (IS) group.'

In [10]:
assert len(train_src_clean) == len(train_tgt_clean)

In [11]:
len(labels)

203575

In [12]:
import numpy as np

In [13]:
labels = np.array(labels)
np.save('train_weights.npy', labels)    # .npy extension is added if not given

In [15]:
labels[:12]

array([0.1, 0.9, 0.1, 0.1, 0.1, 0.9, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

In [14]:
# directory = '/home/ml/users/cadencao/XSum/clean_files/'

# with open(directory + 'train.source', 'w', encoding='utf-8') as sf:
#     with open(directory + 'train.target', 'w', encoding='utf-8') as tf:
#         for s, t in zip(train_src_clean, train_tgt_clean):
#             sf.write(s + '\n')
#             tf.write(t + '\n')